In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.common.keys import keys

In [2]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\16478\.wdm\drivers\chromedriver\win32\93.0.4577.15\chromedriver.exe] found in cache


In [3]:
headers = {'Use-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'}
url = 'https://sh.lianjia.com/chengjiao/'
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.text, 'html.parser')
source = soup.find_all('div',{'class' : 'info'})

In [4]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://sh.lianjia.com/chengjiao/")
#link = driver.find_element_by_link_text("下一页")

In [5]:
from selenium.webdriver.common.by import By

In [6]:
source_info = []


for i in source:
        description = {}
        title = i.find('div', class_='title').text
        deal_date = i.find('div', class_='dealDate').text
        desp_1 = i.find('div', class_='houseInfo').text
        desp_2 = i.find('div', class_='totalPrice').text
        unit_price = i.find('div', class_='unitPrice').text
        desp_3 = i.find('div', class_='positionInfo').text
        try:
            desp_4 = i.find('div', class_='dealHouseInfo').text
        except:
            desp_4 = "NaN"
        try:
            desp_5 = i.find('div', class_='dealCycleeInfo').text
        except:
            desp_5 = "Nan"
        try:
            desp_6 = i.find('div', class_='agentInfoList').text
        except:
            desp_6 = "Nan"
                 
        description['Name'] = title
        description['Deal Date'] = deal_date
        description['Orientation'] = desp_1
        description['Total Price'] = desp_2
        description['Unit Price'] = unit_price
        description['Floor'] = desp_3
        description['Subway'] = desp_4
        description['Deal Price / Deal Date'] = desp_5
        description['Agent'] = desp_6
        
        source_info.append(description)   


In [7]:
for i in range(1,2):
    if i > 0:
        URL = url + 'pg' + str(i)
        browser.visit(URL)
        #html = browser.html
        #soup = BeautifulSoup(html, 'html.parser')
        #links = soup.findAll('a', {'class':'img'})

In [8]:
df = pd.DataFrame(source_info)
df.head()

,Name,Deal Date,Orientation,Total Price,Unit Price,Floor,Subway,Deal Price / Deal Date,Agent
0,蔚蓝城市花园 3室2厅 123.31平米,2021.05.28,南 | 精装,1128万,91477元/平,低楼层(共11层) 2004年建板楼,NaN,挂牌1190万成交周期41天,王毅免费咨询
1,高境一村 1室1厅 42.54平米,2021.05.28,南 | 简装,219万,51481元/平,高楼层(共6层) 1995年建板楼,近地铁,挂牌223万成交周期191天,李鹏飞免费咨询
2,开鲁路382弄 1室1厅 41.54平米,2021.05.28,南 | 简装,245万,58980元/平,中楼层(共6层) 1989年建板楼,近地铁,挂牌245万成交周期6天,董亭苇免费咨询
3,保利铃兰公馆 3室2厅 119.87平米,2021.05.28,南 北 | 精装,750万,62568元/平,低楼层(共3层) 2018年建板楼,NaN,挂牌810万成交周期11天,赵梦怡免费咨询
4,馨佳园(菊盛路851弄) 2室2厅 75.85平米,2021.05.28,南 | 精装,361万,47594元/平,中楼层(共18层) 2011年建板楼,NaN,挂牌368万成交周期16天,程玲免费咨询


In [9]:
#house = df['Name'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)')


In [10]:
split_1 = df['Name'].str.split(' ', n=1, expand = True).rename(columns = {0:"House Name",1:'rooms'})
split_1.head()

,House Name,rooms
0,蔚蓝城市花园,3室2厅 123.31平米
1,高境一村,1室1厅 42.54平米
2,开鲁路382弄,1室1厅 41.54平米
3,保利铃兰公馆,3室2厅 119.87平米
4,馨佳园(菊盛路851弄),2室2厅 75.85平米


In [11]:
split_2 = split_1['rooms'].str.split(' ', n=1, expand = True).rename(columns = {0:"Rooms",1:'Square Meters'})
split_2.head()

,Rooms,Square Meters
0,3室2厅,123.31平米
1,1室1厅,42.54平米
2,1室1厅,41.54平米
3,3室2厅,119.87平米
4,2室2厅,75.85平米


In [12]:
room_1 = split_2['Rooms'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)').rename(columns = {0:"Bedroom",1:'living Room'})
room_2 = room_1['living Room'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)').rename(columns = {0:"Living Room",1:'unit'})
room_2.head()

,Living Room,unit
0,2,厅
1,1,厅
2,1,厅
3,2,厅
4,2,厅


In [13]:
merge = split_1.merge(split_2, left_index=True, right_index=True)
merge_2 = merge.merge(room_1, left_index=True, right_index=True)
merge_3 = merge_2.merge(room_2, left_index=True, right_index=True)
merge_4 = merge_3.drop(['rooms','Rooms','living Room','unit'],axis = 1)
merge_4.head()

,House Name,Square Meters,Bedroom,Living Room
0,蔚蓝城市花园,123.31平米,3,2
1,高境一村,42.54平米,1,1
2,开鲁路382弄,41.54平米,1,1
3,保利铃兰公馆,119.87平米,3,2
4,馨佳园(菊盛路851弄),75.85平米,2,2


In [14]:
split_3 = merge_4['Square Meters'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)').rename(columns = {0:"Size (m2)",1:'unit'})
split_4 = split_3.drop(['unit'],axis = 1)
House_Info = merge_4.merge(split_4, left_index=True, right_index=True)
House_Info.head()

,House Name,Square Meters,Bedroom,Living Room,Size (m2)
0,蔚蓝城市花园,123.31平米,3,2,123.31
1,高境一村,42.54平米,1,1,42.54
2,开鲁路382弄,41.54平米,1,1,41.54
3,保利铃兰公馆,119.87平米,3,2,119.87
4,馨佳园(菊盛路851弄),75.85平米,2,2,75.85


In [15]:
price = df['Total Price'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)').rename(columns = {0:"Total Price (￥10,000)",1:'Unit'})
price_2 = price.drop(['Unit'], axis = 1)
House_Info_2 = House_Info.merge(price_2, left_index=True, right_index=True)
House_Info_2.head()

,House Name,Square Meters,Bedroom,Living Room,Size (m2),"Total Price (￥10,000)"
0,蔚蓝城市花园,123.31平米,3,2,123.31,1128
1,高境一村,42.54平米,1,1,42.54,219
2,开鲁路382弄,41.54平米,1,1,41.54,245
3,保利铃兰公馆,119.87平米,3,2,119.87,750
4,馨佳园(菊盛路851弄),75.85平米,2,2,75.85,361


In [16]:
price_3 = df['Unit Price'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)').rename(columns = {0:"Unit Price (￥/m2)",1:'Unit'})
price_4 = price_3.drop(['Unit'], axis = 1)
House_Info_3 = House_Info_2.merge(price_4, left_index=True, right_index=True)
House_Info_3.head()


,House Name,Square Meters,Bedroom,Living Room,Size (m2),"Total Price (￥10,000)",Unit Price (￥/m2)
0,蔚蓝城市花园,123.31平米,3,2,123.31,1128,91477
1,高境一村,42.54平米,1,1,42.54,219,51481
2,开鲁路382弄,41.54平米,1,1,41.54,245,58980
3,保利铃兰公馆,119.87平米,3,2,119.87,750,62568
4,馨佳园(菊盛路851弄),75.85平米,2,2,75.85,361,47594


In [17]:
def change_string(i):
    if i == "近地铁":
        return 1
    else:
        return 0

House_Info_3['Subway'] = df['Subway'].apply(change_string)
House_Info_3.head(10)

,House Name,Square Meters,Bedroom,Living Room,Size (m2),"Total Price (￥10,000)",Unit Price (￥/m2),Subway
0,蔚蓝城市花园,123.31平米,3,2,123.31,1128,91477,0
1,高境一村,42.54平米,1,1,42.54,219,51481,1
2,开鲁路382弄,41.54平米,1,1,41.54,245,58980,1
3,保利铃兰公馆,119.87平米,3,2,119.87,750,62568,0
4,馨佳园(菊盛路851弄),75.85平米,2,2,75.85,361,47594,0
5,洛川中路1100弄,45.65平米,2,1,45.65,440,96386,0
6,虹梅景苑一街坊,68.74平米,1,2,68.74,361,52517,0
7,海韵茗庭,112平米,2,2,112,837,74733,0
8,海东公寓,88.01平米,2,1,88.01,603,68515,0
9,当代万国府MOMA,53.55平米,1,1,53.55,366,68348,0


In [18]:
size = House_Info_3['Square Meters'].str.extract(r'([\x00-\x7F]+)([\u4e00-\u9fff]+.*$)').rename(columns = {0:'Size (m2)', 1:'Unit'})
size_2 = size.drop(['Unit'], axis = 1)
House_Info_Final = House_Info_3.merge(size_2, left_index=True, right_index=True).drop(['Square Meters'],axis = 1)
House_Info_Final.head(10)

,House Name,Bedroom,Living Room,Size (m2)_x,"Total Price (￥10,000)",Unit Price (￥/m2),Subway,Size (m2)_y
0,蔚蓝城市花园,3,2,123.31,1128,91477,0,123.31
1,高境一村,1,1,42.54,219,51481,1,42.54
2,开鲁路382弄,1,1,41.54,245,58980,1,41.54
3,保利铃兰公馆,3,2,119.87,750,62568,0,119.87
4,馨佳园(菊盛路851弄),2,2,75.85,361,47594,0,75.85
5,洛川中路1100弄,2,1,45.65,440,96386,0,45.65
6,虹梅景苑一街坊,1,2,68.74,361,52517,0,68.74
7,海韵茗庭,2,2,112,837,74733,0,112
8,海东公寓,2,1,88.01,603,68515,0,88.01
9,当代万国府MOMA,1,1,53.55,366,68348,0,53.55


In [19]:
House_Info_Final.to_csv('Resources/Provisional database.csv', index = False)